In [1]:
import random
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import Aer
from qiskit.circuit.library import HGate

numberOfSinglets = 10000

qr = QuantumRegister(2, 'q')
cr = ClassicalRegister(2, 'c')

singlet = QuantumCircuit(qr, cr, name="Singlet")
singlet.h(qr[0])
singlet.cx(qr[0], qr[1])
singlet.x(qr[1])
singlet.z(qr[0])

prover_operation = HGate()
verifier_operation = HGate()

x = [random.randint(0, 1) for _ in range(numberOfSinglets)]
y = [random.randint(0, 1) for _ in range(numberOfSinglets)]

a = [random.randint(0, 1) for _ in range(numberOfSinglets)]
b = [random.randint(0, 1) for _ in range(numberOfSinglets)]

def apply_x_measurement_basis(circuit, qubit, basis_choice):
    if basis_choice == 1:
        circuit.h(qubit)

def apply_y_measurement_basis(circuit, qubit, basis_choice):
    if basis_choice == 0:
        circuit.ry(-np.pi/4, qubit)
    else:
        circuit.ry(np.pi/4, qubit)

circuits = []

for i in range(numberOfSinglets):
    combined_circuit = singlet.copy(name=f"Singlet_{i}:A{x[i]}_B{y[i]}")
    
    if b[i] == 1:
        combined_circuit = combined_circuit.compose(prover_operation, qubits=[qr[1]])
    if a[i] == 1:
        combined_circuit = combined_circuit.compose(verifier_operation, qubits=[qr[1]])
    
    apply_x_measurement_basis(combined_circuit, qr[0], x[i])
    apply_y_measurement_basis(combined_circuit, qr[1], y[i])
    
    combined_circuit.measure(qr, cr)
    circuits.append(combined_circuit)

aer_sim = Aer.get_backend("aer_simulator")
compiled_circuits = transpile(circuits, aer_sim)
job = aer_sim.run(compiled_circuits, shots=1, memory=True)
result = job.result()

ModeAResults = []
ModeBResults = []

for i in range(numberOfSinglets):
    res = result.get_memory(i)[0]
    index = int(res, 2)

    if index == 0:
        ModeAResults.append(+1)
        ModeBResults.append(+1)
    elif index == 1:
        ModeAResults.append(+1)
        ModeBResults.append(-1)
    elif index == 2:
        ModeAResults.append(-1)
        ModeBResults.append(+1)
    elif index == 3:
        ModeAResults.append(-1)
        ModeBResults.append(-1)

def chsh_corr(result, x, y, numberOfSinglets):
    countA0B0 = [0, 0, 0, 0]
    countA0B1 = [0, 0, 0, 0]
    countA1B0 = [0, 0, 0, 0]
    countA1B1 = [0, 0, 0, 0]

    for i in range(numberOfSinglets):
        res = result.get_memory(i)[0]
        index = int(res, 2)

        if x[i] == 0 and y[i] == 0:
            countA0B0[index] += 1
        elif x[i] == 0 and y[i] == 1:
            countA0B1[index] += 1
        elif x[i] == 1 and y[i] == 0:
            countA1B0[index] += 1
        elif x[i] == 1 and y[i] == 1:
            countA1B1[index] += 1

    def expectation(counts):
        total = sum(counts)
        if total == 0:
            return 0
        return (counts[0] - counts[1] - counts[2] + counts[3]) / total

    expect00 = expectation(countA0B0)
    expect01 = expectation(countA0B1)
    expect10 = expectation(countA1B0)
    expect11 = expectation(countA1B1)

    corr = expect00 + expect01 + expect10 - expect11
    return corr

corr = chsh_corr(result, x, y, numberOfSinglets)
print("CHSH correlation value:", round(corr, 3))


CHSH correlation value: -1.463
